In [ ]:
import praw
import json
from datetime import datetime, timezone
from dotenv import load_dotenv, dotenv_values 

from tqdm import tqdm

import pandas as pd
import os
from scraper import RedditScraper
load_dotenv()

True

In [34]:
client_id = os.getenv("CLIENT_ID")
client_secret=os.getenv("CLIENT_SECRET")
user_agent=os.getenv("USER_AGENT")
scraper = RedditScraper(client_id, client_secret, user_agent)

## User comment fetch

In [35]:
subreddits = [
    "books"
] 

In [36]:
BASE_DIR = os.path.abspath(os.path.join(os.getcwd(), "..", "..", "Data"))
USERLIST_FILEPATH = os.path.join(BASE_DIR, 'Reddit Scrape/unique_users.csv')
USERCOMM_SAVEPATH = os.path.join(BASE_DIR, 'usercomments')

os.makedirs(USERCOMM_SAVEPATH, exist_ok=True)

In [ ]:
users_df = pd.read_csv(USERLIST_FILEPATH)
users_list = users_df['user'].tolist()

10


In [38]:
def pull_user_comments(userslist, subreddits, save_basepath):

    for user in tqdm(userslist, desc="Scraping user comments"):
        save_file = f'{user}_usercomments.json'
        save_path = os.path.join(save_basepath, save_file)

        posts = scraper.scrape_user_political_comments(user, subreddits)

        with open(save_path, "w") as f:
            json.dump(posts, f, indent=4)

        # Print progress for debugging/logging (optional)
        print(f"Comments saved to {save_path}")

In [ ]:
pull_user_comments(users_list[10:], subreddits, USERCOMM_SAVEPATH)

Scraping user comments:   0%|          | 0/29901 [00:00<?, ?it/s]

{'greentext', 'comedyheaven', 'tumblr', 'LifeProTips', 'memesopdidnotlike', 'Zillennials', 'movies', 'toptalent', 'intermittentfasting', 'starterpacks', 'ihadastroke', 'reactiongifs', 'StrangerThings', 'buffy', 'muacjdiscussion', 'BetterEveryLoop', 'popculturechat', 'TwoXChromosomes', 'NoStupidQuestions', 'DivinityOriginalSin', 'ThatsInsane', 'science', 'confidentlyincorrect', 'PoliticalHumor', 'place', 'TheWayWeWere', 'HolUp', 'TrueOffMyChest', 'Adulting', 'DownvotedToOblivion', 'insanepeoplefacebook', 'criticalrole', 'Futurology', 'awfuleverything', 'nyc', 'AskReddit', 'CozyPlaces', 'AdviceAnimals', 'facepalm', 'oddlysatisfying', 'Whatcouldgowrong', 'IncelTears', 'cursedcomments', 'pcmasterrace', 'wholesomememes', 'antiwork', 'instant_regret', 'newyork', 'gaming', 'WhitePeopleTwitter', 'Piracy', 'RomanceBooks', 'Damnthatsinteresting', 'WatchPeopleDieInside', 'SubredditDrama', 'Showerthoughts', 'NotHowGirlsWork', 'LateStageCapitalism', 'mildlyinteresting', 'itookapicture', 'fansofcrit

Scraping user comments:   0%|          | 1/29901 [00:32<272:02:42, 32.75s/it]

Comments saved to /Users/smutnuri/Documents/UVA/_Projects/Fall24/CBM/CBMRepo/Data/usercomments/-BetchPLZ_usercomments.json
{'mtg', 'notinteresting', 'Jungle_Mains', 'greentext', 'tumblr', 'Funnymemes', 'LifeProTips', 'gadgets', 'EDH', 'AITAH', 'college', 'starterpacks', 'terriblefacebookmemes', 'fuckcars', 'ADCMains', 'NoStupidQuestions', 'MTGO', 'science', 'chess', 'Anticonsumption', '196', 'LinusTechTips', 'Adulting', 'DownvotedToOblivion', 'MarkMyWords', 'yugioh', 'ProgrammerHumor', 'nvidia', 'redditmoment', 'AskReddit', 'AMA', 'ModernMagic', 'stunfisk', 'SSBPM', 'facepalm', 'Whatcouldgowrong', 'cats', 'The10thDentist', 'CollegeRant', 'pcmasterrace', 'TopCharacterDesigns', 'BikiniBottomTwitter', 'grayandwhitecats', 'gaming', 'WhitePeopleTwitter', 'WatchPeopleDieInside', 'adventofcode', 'buildapc', 'Damnthatsinteresting', 'esports', 'cardgames', 'magicTCG', 'mildlyinteresting', 'nostalgia', 'geography', 'rareinsults', 'smashbros', 'shitposting', 'technology', 'kaisamains', 'Competiti

Scraping user comments:   0%|          | 2/29901 [00:50<197:21:36, 23.76s/it]

...

Comments saved to /Users/smutnuri/Documents/UVA/_Projects/Fall24/CBM/CBMRepo/Data/usercomments/102aksea102_usercomments.json


Scraping user comments:   0%|          | 94/29901 [19:25<80:58:09,  9.78s/it] 

{'grammar', 'books'}
Comments saved to /Users/smutnuri/Documents/UVA/_Projects/Fall24/CBM/CBMRepo/Data/usercomments/10EE22_usercomments.json
{'OpenAI', 'AskMenOver30', 'mildlyinteresting', 'ScienceFictionBooks', 'buffy', 'TwoHotTakes', 'popculturechat', 'FromTVEpix', 'BaldursGate3', 'shittytattoos', 'ExtremeHorrorLit', 'television', 'questions', 'rva', 'sitcoms', 'Frostpunk', 'FromTVShow', 'Watches', 'books', 'MarkMyWords', 'SiloTVSeries', 'BestofRedditorUpdates', 'AskMenAdvice', 'RVADandD', 'OldSchoolCool', 'AskReddit', 'AMA', 'beatles', 'ChatGPTJailbreak', 'FromSeries', 'AmIOverreacting', 'suggestmeabook'}


Scraping user comments:   0%|          | 95/29901 [19:26<59:15:53,  7.16s/it]

Comments saved to /Users/smutnuri/Documents/UVA/_Projects/Fall24/CBM/CBMRepo/Data/usercomments/10Hoursofsleepforme_usercomments.json
{'findareddit', 'sweden', 'adhdwomen', 'GetMotivated', 'fictionalpsychology', 'sex', 'ADHD', 'swedishproblems', 'Hair', 'raisedbynarcissists', 'makemychoice', 'Eldenring', 'ManHands', 'CrossStitch', 'imaginarymaps', 'NoStupidQuestions', 'AskWomenNoCensor', 'Warhammer40k', 'Svenska', 'aww', 'lotr', 'EldenBling', 'RandomThoughts', 'AskMen', 'Embroidery', 'AmItheAsshole', 'GenZ', 'tifu', 'pureasoiaf', 'PixelArt', 'Stormlight_Archive', 'voyager', 'books', 'BestofRedditorUpdates', 'SkincareAddiction', 'CasualConversation', 'MadeMeSmile', 'darksouls', 'AskReddit', 'harrypotter', 'Catio', 'AskWomen', 'femalehairadvice', 'SubSanctuary', 'AskDocs', 'EldenRingLoreTalk', 'Cosmere', 'darksouls3', 'AskWomenOver30', 'wholesomememes', 'offmychest', 'DesignMyRoom', 'AccidentalRenaissance', 'cremposting', 'gaming', 'SoulsSliders'}


## Post fetch

In [ ]:
# post_url = "https://www.reddit.com/r/politics/comments/1gc7k0y/bcso_voter_punches_poll_worker_when_reminded/"

# # Retrieve comments
# comments = scraper.get_comments_from_post(post_url)

# # Save comments to a JSON file
# with open("../Data/comments_graph.json", "w") as f:
#     json.dump(comments, f, indent=4)

# print("Comments saved to comments.json")